In [11]:
# Exercise 11-1

# Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth.
# Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? 
# You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.


In [6]:
import first
import statsmodels.formula.api as smf
import nsfg
import pandas as pd
import numpy as np

In [2]:
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [3]:
# variables I found that have a statistically significant effect on pregnancy length.

model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Sun, 13 Feb 2022   Prob (F-statistic):           5.09e-22
Time:                        15:20:37   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Exercise 11-3

# If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels 
# with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how 
# many children a woman has born; in the NSFG dataset, this variable is called numbabes.

# Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income 
# exceeds $75,000. How many children would you predict she has born?
# Answer- 2.5 children

In [4]:
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join['agepreg_2'] = join.agepreg**2

C:\Users\bland\AppData\Local\Temp/ipykernel_15148/943351468.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  join['agepreg_2'] = join.agepreg**2


In [8]:
# Here is the model that worked the best.

formula = ('numbabes ~ agepreg + agepreg_2+ C(race) + '
              'totincr + hieduc')
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.706029
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8877
Method:                           MLE   Df Model:                            6
Date:                Sun, 13 Feb 2022   Pseudo R-squ.:                 0.02019
Time:                        15:25:22   Log-Likelihood:                -15156.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.092e-131
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.4241      0.118     12.039      0.000       1.192       1.656
C(race)[T.2]    -0.1572      0.015    -10.567      0.000      -0.186      -0.128
C(race)[T.3]    -0.1254      0.025     -5.101      0.000      -0.174      -0.077
agepreg          0.0097      0.009      1.042      0.297      -0.009       0.028
agepreg_2     2.015e-05      0.000      0.114      0.909      -0.000       0.000
totincr         -0.0140      0.002     -7.395      0.000      -0.018      -0.010
hieduc          -0.0508      0.003    -15.776      0.000      -0.057      -0.045
================================================================================
"""

In [9]:
columns = ['agepreg', 'agepreg_2', 'race', 'totincr', 'hieduc']
new = pd.DataFrame([[35, 35**2, 1, 14, 13]], columns=columns)
results.predict(new)

0    2.541524
dtype: float64

In [ ]:
# Exercise 11-4

# If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implementedin 
# StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, 
# widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called rmarital.

# Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about 
# $45,000. What is the probability that she is married and cohabitating?

# Answer - There is about a 70% chance of being married and .05% chance of not being married and just cohabitating. 
# We have a .005% widowed, 15% divorced, .04% separated, and .05% never married


In [10]:
# Here is the model that worked the best.

formula = ('rmarital ~ agepreg + agepreg_2 + C(race) + '
              'totincr + hieduc')
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.110513
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Sun, 13 Feb 2022   Pseudo R-squ.:                  0.1479
Time:                        15:29:11   Log-Likelihood:                -9865.8
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        5.0097      0.704      7.116      0.000       3.630       6.389
C(race)[T.2]    -0.7612      0.088     -8.639      0.000      -0.934      -0.588
C(race)[T.3]    -0.4743      0.135     -3.525      0.000      -0.738      -0.211
agepreg         -0.2256      0.057     -3.970      0.000      -0.337      -0.114
agepreg_2        0.0030      0.001      2.687      0.007       0.001       0.005
totincr         -0.1441      0.011    -12.623      0.000      -0.166      -0.122
hieduc          -0.1434      0.018     -7.756      0.000      -0.180      -0.107
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.1395      1.583      2.615      0.009       1.036       7.243
C(race)[T.2]    -0.5580      0.235     -2.376      0.018      -1.018      -0.098
C(race)[T.3]    -0.1485      0.331     -0.449      0.653      -0.797       0.500
agepreg         -0.3812      0.122     -3.117      0.002      -0.621      -0.141
agepreg_2        0.0073      0.002      3.259      0.001       0.003       0.012
totincr         -0.3094      0.032     -9.792      0.000      -0.371      -0.248
hieduc          -0.0587      0.048     -1.230      0.219      -0.152       0.035
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.8155      0.796     -1.024      0.306      -2.377       0.745
C(race)[T.2]    -0.3452      0.093     -3.726      0.000      -0.527      -0.164
C(race)[T.3]    -0.8567      0.170     -5.052      0.000      -1.189      -0.524
agepreg          0.0450      0.063      0.720      0.471      -0.078       0.168
agepreg_2       -0.0016      0.001     -1.371      0.170      -0.004       0.001
totincr         -0.2064      0.011    -18.220      0.000      -0.229      -0.184
hieduc           0.1173      0.020      5.919      0.000       0.078       0.156
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.6377      0.901      2.928      0.003       0.872       4.404
C(race)[T.2]    -1.0139      0.104     -9.796      0.000      -1.217      -0.811
C(race)[T.3]    -0.5603      0.156     -3.603      0.000      -0.865      -0.255
agepreg         -0.1039      0.073     -1.430      0.153      -0.246       0.038
agepreg_2        0.0009      0.001      0.600      0.549      -0.002       0.004
totincr         -0.2844      0.014    -19.791      0.000      -0.313      -0.256
hieduc           0.0183      0.023      0.796      0.426      -0.027       0.063
-

In [23]:
# Solution
columns = ['agepreg', 'agepreg_2', 'race', 'totincr', 'hieduc'] 
new = pd.DataFrame([[25, 25**2, 2, 9, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.69576,0.05442,0.005351,0.154058,0.043068,0.047344
